# YOLO
The goal of this notebook is to do transfer learning of Fishial´s already implemented YOLO model. We do this by using Ultralytics

In [2]:
!pip install ultralytics

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 10.5 MB/s  0:00:00
   ---------------------------------------- 0.0/783.6 kB ? eta -:--:--
   ---------------------------------------- 783.6/783.6 kB 11.1 MB/s  0:00:00
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   -- ------------------------------------- 2.4/41.3 MB 12.3 MB/s eta 0:00:04
   ---- ----------------------------------- 4.7/41.3 MB 11.9 MB/s eta 0:00:04
   ------ --------------------------------- 7.1/41.3 MB 11.8 MB/s eta 0:00:03
   --------- ------------------------------ 9.4/41.3 MB 11.7 MB/s eta 0:00:03
   ----------- ---------------------------- 12.1/41.3 MB 11.6 MB/s eta 0:00:03
   ------------- -------------------------- 14.4/41.3 MB 11.8 MB/s eta 0:00:03
   ---------------- ----------------------- 16.8/41.3 MB 11.7 MB/s eta 0:00:03
   ------------------ --------------------- 19.1/41.3 MB 11.6 MB/s eta 0:00:02
   -------

In [3]:
from ultralytics import YOLO
import os
import requests
import logging
from zipfile import ZipFile
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
import copy
from IPython.display import Image, display

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\kiennd\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
DATA_DIR = "data/"
timor_leste_data_path = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth
images_path = os.path.join(DATA_DIR, "timor_leste") # Images

In [ ]:
# Filter out relevant images
relevant_species = ['Alectis ciliaris', 'Aphareus rutilans', 'Caranx ignobilis', 'Caranx lugubris', 'Caranx melampygus', 'Caranx sexfasciatus', 'Chirocentrus dorab', 'Chirocentrus nudus', 'Decapterus macrosoma', 'Elagatis bipinnulata', 'Epinephelus maculatus', 'Epinephelus radiatus', 'Etelis carbunculus', 'Gymnocranius grandoculis', 'Katsuwonus pelamis', 'Lethrinus atkinsoni', 'Lethrinus erythracanthus', 'Lethrinus obsoletus', 'Lethrinus ornatus', 'Lutjanus bohar', 'Lutjanus fulviflamma', 'Lutjanus fulvus', 'Lutjanus gibbus', 'Lutjanus johnii', 'Lutjanus kasmira', 'Lutjanus rivulatus', 'Lutjanus russellii', 'Lutjanus timoriensis', 'Monotaxis grandoculis', 'Psettodes erumei', 'Rastrelliger kanagurta', 'Sardinella albella', 'Scolopsis lineata', 'Scolopsis vosmeri', 'Scomberoides lysan', 'Scomberomorus commerson', 'Seriola dumerili', 'Variola albimarginata']

# Read annotation info (Timor-leste)
df_tl_ann = pd.read_csv(timor_leste_data_path, encoding="utf-8-sig", header=0, skiprows=1)
df_tl_ann = df_tl_ann[["image_file", "catch_name_en", "Species_name", "Family"]]

# Filter by species in relevant_species
df_filtered = df_tl_ann[df_tl_ann["Species_name"].isin(relevant_species)]

# Keep only images that actually exist
existing_files = set(os.listdir(images_path))

df_filtered = df_filtered[df_filtered["image_file"].isin(existing_files)]

# Convert to list of filenames
filtered_images= df_filtered["image_file"].tolist()

print("Total annotated images:", len(df_tl_ann))
print("Relevant images", len(df_filtered))
print("Unique final JPGs:", len(set(filtered_images)))


Total annotated images: 603
Relevant images 248
Unique final JPGs: 217


In [ ]:
# Load a pretrained YOLO11n model
model = YOLO("yolo11n-cls.pt")

# Train the model
train_results = model.train(
    data=DATA_DIR+"dataset",
    epochs=50,
    imgsz=640,
    device="cuda",
)

# Evaluate the model's performance on the validation set
metrics = model.val()

display(Image(filename="runs/classify/train/results.png"))
display(Image(filename="runs/classify/val/confusion_matrix.png"))

# Perform object detection on an image
for image in filtered_images[:10]:
  results = model(DATA_DIR + "/timor_leste/" + image)
  results[0].show()

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/FISHIAL/dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=T

KeyboardInterrupt: 

# Dataset

In [ ]:
images_path = os.path.join(DATA_DIR, "timor_leste") # Images
timor_leste_data_path = os.path.join(DATA_DIR, "timor-leste.csv") # Annotation info for ground truth

In [ ]:
# Read annotation info (Timor-leste)
df_tl_ann = pd.read_csv(timor_leste_data_path, encoding="utf-8-sig", header=0, skiprows=1)
df_tl_ann = df_tl_ann[["image_file", "catch_name_en", "Species_name", "Family"]]

In [ ]:
raw = """

"""

classes = [line.strip() for line in raw.splitlines() if line.strip()]

# 2) Format: "Genus species" (first word capitalized, others lowercase)
def format_name(name: str) -> str:
    # Leave codes / single tokens as-is
    if "_" not in name:
        return name

    parts = name.split("_")
    parts[0] = parts[0].capitalize()          # Genus
    parts[1:] = [p.lower() for p in parts[1:]]  # species etc.
    return " ".join(parts)

input_species = [format_name(n) for n in classes]


In [ ]:
registered_species = set(df_tl_ann["catch_name_en"].dropna().astype(str).str.strip().unique())

input_species = [
    "Blacksaddle filefish",
    "Blue streak cleaner wrasse",
    "Butterfly fish",
    "Checkerboard wrasse",
    "Clark's anemonefish",
    "Commerson's frogfish",
    "Lionfish",
    "Longfin bannerfish",
    "Manybar goatfish",
    "Moorish idol",
    "Pyramid butterflyfish",
    "Redtoothed triggerfish",
    "Reticulate dascyllus",
    "Ribboned sweetlips",
    "Sea goldie",
    "Sergeant major fish",
    "Teira batfish",
    "Weber's chromis"
]

input_species = set([s.strip() for s in input_species])

# Compare sets
intersection = registered_species.intersection(input_species)
only_in_input = input_species - registered_species
only_in_registered = registered_species - input_species

# Print results
print("Species found in BOTH input and dataset:")
print(sorted(intersection), "\n")

print("Species in INPUT but NOT in dataset:")
print(sorted(only_in_input), "\n")

print("Species in dataset but NOT in input list:")
print(f"(Showing first 20) {sorted(list(only_in_registered))[:20]}")


Species found in BOTH input and dataset:
[] 

Species in INPUT but NOT in dataset:
['Blacksaddle filefish', 'Blue streak cleaner wrasse', 'Butterfly fish', 'Checkerboard wrasse', "Clark's anemonefish", "Commerson's frogfish", 'Lionfish', 'Longfin bannerfish', 'Manybar goatfish', 'Moorish idol', 'Pyramid butterflyfish', 'Redtoothed triggerfish', 'Reticulate dascyllus', 'Ribboned sweetlips', 'Sea goldie', 'Sergeant major fish', 'Teira batfish', "Weber's chromis"] 

Species in dataset but NOT in input list:
(Showing first 20) ['Barracuda & Jacks/Trevally/Other Scad', 'Barracuda & Jacks/Trevally/Other Scad & Jacks/Trevally/Other Scad', 'Barracuda & Jacks/Trevally/Other Scad & Snapper/seaperch', 'Bream & Emperor & Snapper/seaperch & Unknown & Unknown', 'Bream & Grouper & Unknown', 'Bream & Jacks/Trevally/Other Scad', 'Bream & Jacks/Trevally/Other Scad & Stingrays', 'Bream & Snapper/seaperch', 'Emperor', 'Emperor & Goatfish & Spinefoot & Tuna/Bonito/Other Mackerel', 'Emperor & Jacks/Trevally

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

# Load the TorchScript model
model = torch.jit.load('model.ts')

# Put model in training mode
model.train()

# Set up your data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

# Load your new image dataset
train_dataset = datasets.ImageFolder('path/to/new/images', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Set up optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Save the fine-tuned model
torch.jit.save(model, 'finetuned_model.ts')